In [ ]:
import os, sys, numpy as np, torch, matplotlib.pyplot as plt, cv2
sys.path.append(os.path.abspath(".."))

from core.model import TumorNet34Bayes
from core.utils import GradCAMpp
from contextlib import contextmanager

In [ ]:
ROOT_DIR = os.path.abspath("..")
DATA_DIR = os.path.join(ROOT_DIR, "data", "processed")
CKPT_DIR = os.path.join(ROOT_DIR, "results", "checkpoints")
VIS_DIR  = os.path.join(ROOT_DIR, "results", "visualizations")
os.makedirs(VIS_DIR, exist_ok=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = TumorNet34Bayes(dropout_p=0.4).to(device)
model.load_state_dict(torch.load(
    os.path.join(CKPT_DIR, "model_weights_best.pth"),  
    map_location=device
))
model.eval()

In [ ]:
X_test = np.load(os.path.join(DATA_DIR, "X_test.npy"))
y_test = np.load(os.path.join(DATA_DIR, "y_test.npy"))
X_test_t = torch.tensor(X_test, dtype=torch.float32).permute(0,3,1,2).to(device)

In [ ]:
idx = 0
x = X_test_t[idx:idx+1]

gc = GradCAMpp(model, target_layer="layer4")
model.zero_grad(set_to_none=True)

In [ ]:
logits, conf, probs = model(x)      
logits.sum().backward()
cam = gc.generate(logits).squeeze(0).detach().cpu().numpy()
gc.remove()

In [ ]:
base_img = (X_test[idx] * 255).astype(np.uint8)
cam_up = cv2.resize(cam, (base_img.shape[1], base_img.shape[0]))
plt.imshow(base_img)
plt.imshow(cam_up, cmap="jet", alpha=0.5)
plt.title(f"Grad-CAM++ | Label={int(y_test[idx])}")
plt.axis("off")
plt.savefig(os.path.join(VIS_DIR, "gradcampp_example.png"))
plt.show()

In [ ]:
@contextmanager
def enable_dropout(model):
    """Context manager to enable dropout during inference."""
    training_states = {}
    for m in model.modules():
        if isinstance(m, torch.nn.Dropout):
            training_states[m] = m.training
            m.train()
    try:
        yield
    finally:
        for m, state in training_states.items():
            m.train(state)

In [ ]:
@torch.no_grad()
def mc_pass(model, x, T=20):
    """MC Dropout forward pass safe for batch size 1."""
    preds = []
    with enable_dropout(model):  
        for _ in range(T):
            logits, probs, conf = model(x) 
            preds.append(torch.sigmoid(logits))
    preds = torch.stack(preds, dim=0)
    return preds.mean(0).squeeze(1), preds.std(0).squeeze(1)

In [ ]:
idx = 1
x = X_test_t[idx:idx+1]
mean_p, epistemic = mc_pass(model, x, T=20)

In [ ]:
plt.imshow(X_test[idx])
plt.title(f"MC Dropout: p={float(mean_p.item()):.2f}, epi-σ={float(epistemic.item()):.2f}")
plt.axis("off")
plt.savefig(os.path.join(VIS_DIR, "uncertainty_example.png"))
plt.show()